In [ ]:
################ convert pdf to images ########################

from pdf2image import convert_from_path
import os

images_path = os.getcwd()

def pdf_to_images(pdf, filename):
    pdf = pdf.replace('\\', '\\\\')
    # print(pdf)
    try:
        images = convert_from_path(pdf)
        for i in range(len(images)):
            images[i].save(images_path+'\\images\\{}_page{}'.format(filename, i) +'.jpg', 'JPEG')
    except:
        pass

pdf_path = 'C:\\Users\\Lenovo\\Downloads\\Net Zero 1st Set Data - 30-11-2023 2 (1)\\Business Travel Emission Management\\2023\\Hotel Expenses'

#store all the file names in this list
filelist = []

for root, dirs, files in os.walk(pdf_path):
	for file in files:
        #append the file name to the list
		filelist.append(os.path.join(root,file))


# print all the file names
for file in filelist:
    filename = os.path.split(file)[-1].split('.pdf')[0]
    if ' ' in filename:
        filename = filename.replace(' ', '_')
    # print(filename)
    try:
        pdf_to_images(file, filename)
    except:
        pass

In [3]:
########################## parse output of preprocessed images #############################

from PIL import Image, ImageEnhance
import cv2
import numpy as np
from skimage import io
from skimage.transform import rotate
from skimage.color import rgb2gray
from deskew import determine_skew
from matplotlib import pyplot as plt
from pdf2image import convert_from_path
import shutil
import glob
import os

def set_dpi(image, preproc_image):
    im = Image.open(image)
    im.save(preproc_image, dpi=(600,600))
    return preproc_image

def increase_contrast(image, preproc_image):
    im = Image.open(image)
    enhancer = ImageEnhance.Contrast(im)
    factor = 2 #increase contrast
    im_output = enhancer.enhance(factor)
    im_output.save(preproc_image)
    return preproc_image

def increase_sharpness(image, preproc_image):
    im = Image.open(image)
    enhancer = ImageEnhance.Sharpness(im)
    factor = 2
    im_output = enhancer.enhance(factor)
    im_output.save(preproc_image)
    return preproc_image

def convert_to_grayscale(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def convert_to_rgb(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def otsu_threshold(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    # im = cv2.threshold(im, 120, 255, cv2.THRESH_BINARY, cv2.THRESH_OTSU)[1]
    im = cv2.adaptiveThreshold(im,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def remove_noise(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.fastNlMeansDenoising(im, None, 20, 7, 21)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def deskew(image):
    im = io.imread(image)
    angle = determine_skew(im)
    rotated = rotate(im, angle, resize=True) * 255
    io.imsave(image, rotated.astype(np.uint8))
    return image

combinations = ['convert_to_rgb-set_dpi']
# combinations = ['set_dpi-increase_contrast-increase_sharpness']

image_path = os.getcwd()
for image in glob.glob(image_path+'\\images\\*.jpg'):
    # filename = os.path.split(image)[-1].split('.jpg')[0]
    # print(filename)
    # preproc_image = 'preprocessed_ensemble\\'+filename+'.jpg'
    # shutil.copy(image, preproc_image)
    for combination in combinations:
        filename = os.path.split(image)[-1].split('.jpg')[0]
        preproc_image = image_path+'\\preprocessed_images\\'+filename+'.jpg'
        shutil.copy(image, preproc_image)
        methods = combination.split('-')
        for method in methods:
            if 'deskew' in method:
                preproc_image = eval(method+'(preproc_image)')
            else:
                preproc_image = eval(method+'(preproc_image, preproc_image)')
        # print(combination)
        image_name = image_path+'\\preprocessed_images\\'+filename+'_'+combination+'.jpg'
        img = cv2.imread(preproc_image)
        cv2.imwrite(image_name, img)

        psm_values = [4]

        for psm in psm_values:

            parsed_output = image_path+'\\output\\'+filename+'_'+combination+'_'+str(psm)
            command = 'tesseract {} {} -l eng --psm {}'.format(preproc_image, parsed_output, psm)
            os.system(command)

c:\air_ticket\netlink_electricity


In [1]:
import datetime
import re

def extract_date(self, text):
    start_date = ''
    end_date = ''
    for line in text:
        x = re.search('.+from.+to.+', line.lower())
        if x:
            words = line.lower().split()
            ind_from = words.index('from')
            start_date = words[ind_from+1]
            start_date = datetime.datetime.strptime(start_date, '%d/%m/%y').strftime('%Y-%m-%d')
            ind_to = words.index('to')
            end_date = words[ind_to+1]
            end_date = datetime.datetime.strptime(end_date, '%d/%m/%y').strftime('%Y-%m-%d')
    return start_date, end_date

def extract_supplier_name(self, text):
    supplier_name = ''
    if 'annexure' not in text[0].lower():
        supplier_name = text[0]
    return supplier_name

def extract_fuel_consumption(self, text):
    total_fuel = 0
    for i in range(len(text)):
        if 'particulars' in text[i].lower():
            break
    values = []
    for j in range(i+1, len(text)):
        if '--' in text[j].split()[-2] or '-' in text[j].split()[-2]:
            break
        values.append(float(text[j].split()[-2]))
        sum_values = sum(values)
    return sum_values

def extract_fuel_consumption_unit(self):
    return 'kwh'

image_path = os.getcwd()

for file in glob.glob(image_path+'\\output\\*'):
    with open(file, encoding='utf-8') as fp:
        text = fp.read()
        if "arrival" in text.lower() and "departure" in text.lower():
            # print("FILE --> ", file)
            text = text.splitlines()
            text = [line for line in text if not line.isspace() and len(line) > 0]
            
            filename = os.path.split(file)[-1].split('.txt')[0]
            ind = filename.index('page')
            pre = filename[ind+5:]
            filename = filename[:ind+5]

            start_date = extract_date(text)[0]
            end_date = extract_date(text)[1]
            supplier_name = extract_supplier_name(text)
            fuel_cons = extract_fuel_consumption(text)
            fuel_cons_unit = extract_fuel_consumption_unit()
    
    print(file)
    print(start_date)
    print(end_date)
    print(supplier_name)
    print(fuel_cons)
    print(fuel_cons_unit)
    print('-----')